### Индивидуальное задание
Вычислить значение функции $f$ с точностью до сотых в точках из списка $X$.
#### Указание. 
Воспользуйтесь методом evalf() для получения приближенного значения. Значения $f$ в точках из списка $X$ выведите в виде списка.

Устанавливаем numpy, sympy

In [ ]:
!python -m pip install sympy --upgrade
!python -m pip install numpy --upgrade

Импортируем нужные библиотеки

In [ ]:
import numpy as np
import sympy as sp
from sympy.abc import x
from sympy.polys.specialpolys import random_poly
import operator
import os

In [ ]:
%run document_generator.ipynb

Используемые функции (`EX`): 
- `ln(x)`
- `exp(x)`
- `sin(x)`
- `cos(x)`
- `tan(x)`

Используемые внутренние функции (`INNER_EX`) - полиномы 2-ой или 3-ей степени с целочисленными коэффициентами 

Итоговая функция строится следующим образом:  
`f(x) = {+, -}ex1(inner_ex1){+, -, *, /}{+, -}ex2(inner_ex2)`, где:
- `ex1`, `ex2` in `EX`; `ex1 != ex2`
- `inner_ex1`, `inner_ex2` in `INNER_EX`

In [ ]:
EX = [sp.ln, sp.exp, sp.sin, sp.cos, sp.tan]
ARITHMETIC_OPS = [operator.add, operator.sub, operator.mul, operator.truediv]

In [ ]:
def generate_inner_ex():
    return random_poly(x, np.random.randint(2, 4), -10, 10)

In [ ]:
def generate_final_function():
    ex1, ex2 = list(map(lambda ex: np.random.choice([-1, 1]) * ex(generate_inner_ex()), np.random.choice(EX, 2, replace=False)))
    return sp.sympify(np.random.choice(ARITHMETIC_OPS)(ex1, ex2))

In [ ]:
# Restricting potential point values to not get crazy numbers.
POINT_MIN = -1e8
POINT_MAX = 1e8
POINT_EPSILON = 1e-2

In [ ]:
"""
Safely rounds the expression to n decimal points
(see: https://github.com/sympy/sympy/issues/21513).
"""
def safe_round(ex: sp.Expr, n):
    return round(eval(f'{ex.evalf()}'), n)

In [ ]:
"""
Attempts to generate a random list of 3 points, lying within the domain of the given function.
On failure, returns an empty list. 
"""
def generate_points(f: sp.Expr):
    def test(p):
        res = f.subs(x, p) 
        return res.is_finite and res.is_real and POINT_MIN <= res <= POINT_MAX and abs(safe_round(res, 2)) >= POINT_EPSILON

    points = []
    
    # First point is always an integer in [-10, 10].
    first_points = [p for p in range(-10, 11)]
    np.random.shuffle(first_points)

    for p in first_points:
        if test(p):
            points.append(p)
            break
    else:
        return []
    
    # 2 other points are of form: a + b*PI, b != 0
    for point_count in range(2):
        for attempt in range(100):
            p = np.random.randint(-10, 11) + np.random.choice([-1, 1]) * np.random.randint(1, 11) * sp.pi
            if test(p) and p not in points:
                points.append(p)
                break
        else:
            return []
        
    return points

In [ ]:
"""
Returns a pair of a function and 3 points, lying within the domain of it.
"""
def generate_final_function_and_points():
    while True:
        f = generate_final_function()
        ps = generate_points(f)
        if len(ps) > 0:
            return f, ps

In [ ]:
def solve(f: sp.Expr, points):
    return [safe_round(f.subs(x, p), 2) for p in points]

In [ ]:
DESCRIPTION = """Вычислить значение функции $f$ с точностью до сотых в точках из списка $X$.
Указание. Воспользуйтесь методом evalf() для получения приближенного значения. Значения $f$ в точках из списка $X$ выведите в виде списка."""
TASK = """\\begin{{align*}}
    f(x) = {f} && {ps_name} = {ps}
\\end{{align*}}"""

def format_tasks_and_solutions():
    f, ps = generate_final_function_and_points()
    f_latex = sp.latex(f)
    return TASK.format(f = f_latex, ps_name = "X", ps = sp.latex(ps)), TASK.format(f = f_latex, ps_name = "Solutions", ps = sp.latex(solve(f, ps)))

DOC = DocumentGenerator(lambda _: format_tasks_and_solutions(), DESCRIPTION)

In [64]:
write_tasks_and_solutions(DOC, "../out/tasks/task-1.tex", "../out/answers/answer-1.tex", 150)